cài đặt thư viện cần thiết

In [1]:
# Cài đặt hugging face
!pip install transformers
# Cài đặt thư viện underthesea để thực hiện word segment
!pip install underthesea
# Cài đặt pytorch
!pip install torch
# Cài đặt sklearn
!pip install scikit-learn
# thư viên để gọi model có sẵn và xử lý
!pip install torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.4 MB/s eta 0:00:00


In [2]:
import pandas as pd                             # xử lý dữ liệu đọc trong excel
import re                                       # xử lý chuẩn hóa dữ liệu, xóa đi các dấu
from transformers import AutoTokenizer          # tokenize câu để sử dụng PhoBERT
import numpy as np

Chuẩn bị dữ liệu

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_excel('/content/drive/MyDrive/NCKH-2025/data_raw/data_raw.xlsx')

In [5]:
data.columns

Index(['id', 'title', 'sapo', 'content', 'date', 'url', 'category', 'author'], dtype='object')

In [6]:
features = data[['content', 'date']]
features = features.set_index('date')

# dữ liệu để xác định cảm xúc
content_data = features[["content"]]

Chuẩn hóa dữ liệu & Tokenize dữ liệu (yêu cầu của PhoBERT)

In [7]:
def standardize_data(text):
    # Tách câu dựa vào các dấu chấm câu
    sentences = re.split(r'(?<=[.!?]) +', text)
    cleaned_sentences = []

    for sentence in sentences:
        # Xóa dấu câu không cần thiết bên trong câu
        sentence = sentence.replace(",", " ").replace(".", " ") \
            .replace(";", " ").replace("“", " ") \
            .replace(":", " ").replace("”", " ") \
            .replace('"', " ").replace("'", " ") \
            .replace("!", " ").replace("?", " ") \
            .replace("-", " ").replace("?", " ")
        # Xóa khoảng trắng thừa và chuyển về chữ thường
        sentence = sentence.strip().lower()
        cleaned_sentences.append(sentence)

    return cleaned_sentences
content_data['content'] = content_data['content'].apply(standardize_data)

In [8]:
content_data

,content
date,
2021-11-26 09:18:00,[nếu không có gần 5 điểm tăng hôm nay của vic ...
2021-11-26 11:08:00,[hôm nay (26/11) lực bán ròng của khối ngoại ...
2021-11-29 05:11:00,[mặc dù chứng khoán toàn cầu cũng như giá hàng...
2021-11-29 08:38:00,[nhóm blue chips mạnh lên không rõ rệt trong p...
2021-11-30 05:12:00,[hai cổ phiếu trụ quan trọng hôm qua là vic và...
...,...
2025-05-01 14:23:00,[trong năm 2024 vinfast thương hiệu xe điện ...
2025-07-01 15:12:00,[theo báo cáo cập nhật mới nhất vietcap đánh ...
2025-10-01 17:03:00,[ctcp tập đoàn vingroup (mã chứng khoán vic) c...


Tokenize câu văn để chuyển hóa thành vecto

In [9]:
from transformers import AutoTokenizer     # thực hiện load model

# Load PhoBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [10]:
def tokenize_and_encode(content):
    encoded_sentences = []
    for sentence in content:
        # Tokenize và encode từng câu
        encoded_sentence = tokenizer.encode(sentence, truncation=True, padding="max_length", max_length=128)
        encoded_sentences.append(encoded_sentence)
    return encoded_sentences

# Áp dụng cho cột content
content_data['encoded_content'] = content_data['content'].apply(tokenize_and_encode)

In [11]:
content_data

,content,encoded_content
date,,
2021-11-26 09:18:00,[nếu không có gần 5 điểm tăng hôm nay của vic ...,"[[0, 183, 17, 10, 124, 173, 192, 128, 654, 136..."
2021-11-26 11:08:00,[hôm nay (26/11) lực bán ròng của khối ngoại ...,"[[0, 654, 136, 14157, 6670, 7086, 19, 2792, 17..."
2021-11-29 05:11:00,[mặc dù chứng khoán toàn cầu cũng như giá hàng...,"[[0, 1036, 468, 3011, 8804, 668, 630, 32, 42, ..."
2021-11-29 08:38:00,[nhóm blue chips mạnh lên không rõ rệt trong p...,"[[0, 276, 48434, 55033, 384, 72, 17, 297, 1698..."
2021-11-30 05:12:00,[hai cổ phiếu trụ quan trọng hôm qua là vic và...,"[[0, 82, 862, 2210, 3506, 2665, 7594, 654, 89,..."
...,...,...
2025-05-01 14:23:00,[trong năm 2024 vinfast thương hiệu xe điện ...,"[[0, 12, 29, 27035, 10164, 29828, 2057, 3256, ..."
2025-07-01 15:12:00,[theo báo cáo cập nhật mới nhất vietcap đánh ...,"[[0, 63, 441, 9866, 5530, 21697, 60, 67, 18808..."
2025-10-01 17:03:00,[ctcp tập đoàn vingroup (mã chứng khoán vic) c...,"[[0, 1395, 1187, 21146, 834, 639, 1659, 5936, ..."


PhoBERT yêu cầu dữ liệu đầu vào dạng tensor

In [12]:
import torch

# Flatten các câu của từng văn bản thành một tensor
def create_tensors(encoded_content):
    input_ids = [torch.tensor(doc, dtype=torch.long) for doc in encoded_content]
    return input_ids

content_data['input_tensors'] = content_data['encoded_content'].apply(create_tensors)

In [13]:
content_data

,content,encoded_content,input_tensors
date,,,
2021-11-26 09:18:00,[nếu không có gần 5 điểm tăng hôm nay của vic ...,"[[0, 183, 17, 10, 124, 173, 192, 128, 654, 136...","[[tensor(0), tensor(183), tensor(17), tensor(1..."
2021-11-26 11:08:00,[hôm nay (26/11) lực bán ròng của khối ngoại ...,"[[0, 654, 136, 14157, 6670, 7086, 19, 2792, 17...","[[tensor(0), tensor(654), tensor(136), tensor(..."
2021-11-29 05:11:00,[mặc dù chứng khoán toàn cầu cũng như giá hàng...,"[[0, 1036, 468, 3011, 8804, 668, 630, 32, 42, ...","[[tensor(0), tensor(1036), tensor(468), tensor..."
2021-11-29 08:38:00,[nhóm blue chips mạnh lên không rõ rệt trong p...,"[[0, 276, 48434, 55033, 384, 72, 17, 297, 1698...","[[tensor(0), tensor(276), tensor(48434), tenso..."
2021-11-30 05:12:00,[hai cổ phiếu trụ quan trọng hôm qua là vic và...,"[[0, 82, 862, 2210, 3506, 2665, 7594, 654, 89,...","[[tensor(0), tensor(82), tensor(862), tensor(2..."
...,...,...,...
2025-05-01 14:23:00,[trong năm 2024 vinfast thương hiệu xe điện ...,"[[0, 12, 29, 27035, 10164, 29828, 2057, 3256, ...","[[tensor(0), tensor(12), tensor(29), tensor(27..."
2025-07-01 15:12:00,[theo báo cáo cập nhật mới nhất vietcap đánh ...,"[[0, 63, 441, 9866, 5530, 21697, 60, 67, 18808...","[[tensor(0), tensor(63), tensor(441), tensor(9..."
2025-10-01 17:03:00,[ctcp tập đoàn vingroup (mã chứng khoán vic) c...,"[[0, 1395, 1187, 21146, 834, 639, 1659, 5936, ...","[[tensor(0), tensor(1395), tensor(1187), tenso..."


sử dụng model thiết lập sẵn wonrax/phobert-base-vietnamese-sentiment
để phân tích cảm xúc

In [16]:
import torch
from transformers import RobertaForSequenceClassification, AutoTokenizer

# Tải model và tokenizer từ Hugging Face
model_name = "wonrax/phobert-base-vietnamese-sentiment"
model = RobertaForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# Bản đồ nhãn sang cảm xúc
label_mapping = {0: "Tiêu cực", 1: "Trung tính", 2: "Tích cực"}

# Hàm dự đoán cảm xúc cho một câu với log chi tiết
def predict_emotion(sentence_tensor, sentence):
    with torch.no_grad():
        outputs = model(sentence_tensor)
        probs = outputs.logits.softmax(dim=-1)  # Tính xác suất
        predicted_label = torch.argmax(probs, dim=-1).item()  # Lấy nhãn dự đoán
        print(f"Câu: {sentence}")
        print(f"  -> Xác suất: {probs.tolist()}")
        print(f"  -> Cảm xúc: {label_mapping[predicted_label]}")
        return predicted_label, probs.tolist()

# Dự đoán cảm xúc cho từng văn bản
def analyze_sentiments(content_data):
    all_predictions = []

    # Reset lại index để tránh lỗi khi sử dụng index + 1
    content_data = content_data.reset_index()

    final_sentiments = []  # Lưu trữ kết luận cảm xúc của từng văn bản

    for index, row in content_data.iterrows():
        print(f"\nĐang phân tích văn bản tại dòng {index + 1}: {row['content']}")

        sentences = row['encoded_content']  # Danh sách câu đã mã hóa
        sentence_tensor = torch.tensor(sentences)  # Chuyển danh sách các câu thành tensor

        predictions = []
        sentiment_counts = {0: 0, 1: 0, 2: 0}  # Đếm cảm xúc từng câu

        for i, (sentence, encoded_sentence) in enumerate(zip(row['content'], sentence_tensor)):
            print(f"\nCâu {i + 1}: {sentence}")
            label, prob = predict_emotion(encoded_sentence.unsqueeze(0), sentence)  # Dự đoán từng câu
            predictions.append({'sentence': sentence, 'label': label, 'prob': prob})

            # Đếm cảm xúc
            sentiment_counts[label] += 1

        # Kết luận cảm xúc cho văn bản
        overall_sentiment = label_mapping[max(sentiment_counts, key=sentiment_counts.get)]
        print(f"\nTổng hợp cảm xúc cho văn bản: {overall_sentiment}")

        # Lưu lại kết quả
        final_sentiments.append(overall_sentiment)
        all_predictions.append(predictions)

    # Thêm cột kết quả vào content_data
    content_data['sentiment_analysis'] = all_predictions
    content_data['overall_sentiment'] = final_sentiments

    return content_data


# Áp dụng lên dữ liệu
print("Bắt đầu phân tích cảm xúc...")
content_data = analyze_sentiments(content_data)
print("\nHoàn thành phân tích cảm xúc!")

# Hiển thị kết quả rõ ràng
def display_results(content_data):
    for index, row in content_data.iterrows():
        print(f"\nVăn bản tại dòng {index + 1}:")
        for analysis in row['sentiment_analysis']:
            sentence = analysis['sentence']
            label = label_mapping[analysis['label']]
            prob = analysis['prob']
            print(f"  - Câu: {sentence}")
            print(f"    -> Cảm xúc: {label}")
            print(f"    -> Xác suất: {prob}")

# Hiển thị kết quả phân tích
display_results(content_data)


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
    -> Cảm xúc: Tích cực
    -> Xác suất: [[0.046077944338321686, 0.45478299260139465, 0.49913913011550903]]
  - Câu: hpg  vnd  ssi  tcb  dig  tcm  vre  nkg  hdg  vci  pow  evf khớp hàng trăm tỷ đồng mỗi mã và giá giảm rất mạnh 
 
 nhà đầu tư nước ngoài hôm nay ghi nhận bán ròng tổng thể 1 008 8 tỷ đồng trên sàn hose trong đó bán riêng với cổ phiếu vn30 khoảng 660 5 tỷ đồng
    -> Cảm xúc: Tích cực
    -> Xác suất: [[0.0674482136964798, 0.14145256578922272, 0.7910991907119751]]
  - Câu: tuy nhiên do có giao dịch đối ứng trong đợt atc nên tính riêng đợt này  mức bán ròng chỉ khoảng 100 tỷ đồng
    -> Cảm xúc: Trung tính
    -> Xác suất: [[0.009838970378041267, 0.9240345358848572, 0.0661265179514885]]
  - Câu: các cổ phiếu bị xả mạnh là fpt  226 7 tỷ  vnd  129 4 tỷ  vre  85 tỷ  hpg  63 6 tỷ  pow  52 2 tỷ  mwg  42 7 tỷ  vpb  40 9 tỷ… bên mua có evf +61 4 tỷ  ctr +56 4 tỷ  lpb +42 9 tỷ  tch +32 4 tỷ  elc +29 tỷ  pc1 +28 4 tỷ
    -> C

In [17]:
content_data

,date,content,encoded_content,input_tensors,sentiment_analysis,overall_sentiment
0,2021-11-26 09:18:00,[nếu không có gần 5 điểm tăng hôm nay của vic ...,"[[0, 183, 17, 10, 124, 173, 192, 128, 654, 136...","[[tensor(0), tensor(183), tensor(17), tensor(1...",[{'sentence': 'nếu không có gần 5 điểm tăng hô...,Trung tính
1,2021-11-26 11:08:00,[hôm nay (26/11) lực bán ròng của khối ngoại ...,"[[0, 654, 136, 14157, 6670, 7086, 19, 2792, 17...","[[tensor(0), tensor(654), tensor(136), tensor(...",[{'sentence': 'hôm nay (26/11) lực bán ròng c...,Tích cực
2,2021-11-29 05:11:00,[mặc dù chứng khoán toàn cầu cũng như giá hàng...,"[[0, 1036, 468, 3011, 8804, 668, 630, 32, 42, ...","[[tensor(0), tensor(1036), tensor(468), tensor...",[{'sentence': 'mặc dù chứng khoán toàn cầu cũn...,Trung tính
3,2021-11-29 08:38:00,[nhóm blue chips mạnh lên không rõ rệt trong p...,"[[0, 276, 48434, 55033, 384, 72, 17, 297, 1698...","[[tensor(0), tensor(276), tensor(48434), tenso...",[{'sentence': 'nhóm blue chips mạnh lên không ...,Trung tính
4,2021-11-30 05:12:00,[hai cổ phiếu trụ quan trọng hôm qua là vic và...,"[[0, 82, 862, 2210, 3506, 2665, 7594, 654, 89,...","[[tensor(0), tensor(82), tensor(862), tensor(2...",[{'sentence': 'hai cổ phiếu trụ quan trọng hôm...,Trung tính
...,...,...,...,...,...,...
285,2025-05-01 14:23:00,[trong năm 2024 vinfast thương hiệu xe điện ...,"[[0, 12, 29, 27035, 10164, 29828, 2057, 3256, ...","[[tensor(0), tensor(12), tensor(29), tensor(27...",[{'sentence': 'trong năm 2024 vinfast thương...,Trung tính
286,2025-07-01 15:12:00,[theo báo cáo cập nhật mới nhất vietcap đánh ...,"[[0, 63, 441, 9866, 5530, 21697, 60, 67, 18808...","[[tensor(0), tensor(63), tensor(441), tensor(9...",[{'sentence': 'theo báo cáo cập nhật mới nhất ...,Trung tính
287,2025-10-01 17:03:00,[ctcp tập đoàn vingroup (mã chứng khoán vic) c...,"[[0, 1395, 1187, 21146, 834, 639, 1659, 5936, ...","[[tensor(0), tensor(1395), tensor(1187), tenso...",[{'sentence': 'ctcp tập đoàn vingroup (mã chứn...,Trung tính
288,2025-11-01 11:01:00,[ngày 11/01/2025 vinfastcông bố đã bàn giao hơ...,"[[0, 43, 4835, 7818, 6669, 10164, 1866, 22690,...","[[tensor(0), tensor(43), tensor(4835), tensor(...",[{'sentence': 'ngày 11/01/2025 vinfastcông bố ...,Trung tính


In [22]:
# thêm trường nhận diện cảm xúc
data['overall_sentiment'] = content_data['overall_sentiment']

# Save data to an Excel file
output_path = '/content/drive/MyDrive/NCKH-2025/product/data_with_sentiment.xlsx'
data.to_excel(output_path, index=False)

print(f"File saved to {output_path}")


File saved to /content/drive/MyDrive/NCKH-2025/product/data_with_sentiment.xlsx
